In [1]:
import pandas as pd
import requests as rq
import json

In [2]:
api = rq.get("https://api.cartolafc.globo.com/atletas/mercado")

In [3]:
market = json.loads(api.text)

In [4]:
market.keys()

dict_keys(['atletas', 'clubes', 'posicoes', 'status'])

In [5]:
roundN = pd.DataFrame(market["atletas"])

In [6]:
roundN.head(10)

,apelido,atleta_id,clube_id,foto,jogos_num,media_num,nome,pontos_num,posicao_id,preco_num,rodada_id,scout,slug,status_id,variacao_num
0,Vico,95622,284,https://s.glbimg.com/es/sde/f/2018/05/21/8e861...,0,0.00,Marcus Vinicius Aredes Duarte,0.0,5,1.00,6,{},vico,6,0.00
1,Felipe Vizeu,94138,284,https://s.glbimg.com/es/sde/f/2019/04/17/8b7d8...,6,1.00,Felipe dos Reis Pereira Vizeu do Carmo,7.1,5,3.63,6,"{'FC': 4, 'FF': 1, 'FS': 2, 'G': 1, 'PE': 6}",felipe-vizeu,7,1.73
2,Luan,86759,284,https://s.glbimg.com/es/sde/f/2018/05/21/c01a0...,4,3.00,Luan Guilherme de Jesus Vieira,0.0,5,10.15,6,"{'A': 1, 'FC': 2, 'FD': 4, 'FS': 5, 'FT': 1, '...",luan,5,0.00
3,Rodriguez,91251,284,https://s.glbimg.com/es/sde/f/2018/08/09/e6b3f...,1,10.00,Antônio Josenildo Rodrigues de Oliveira,10.0,3,4.03,6,"{'A': 1, 'SG': 1}",rodriguez,7,2.03
4,Marinho,68952,277,https://s.glbimg.com/es/sde/f/2019/05/30/cd8a7...,2,2.60,Mário Sérgio Santos Costa,0.0,5,5.74,6,"{'CA': 1, 'FC': 2, 'FD': 1, 'FF': 2, 'FS': 3, ...",marinho,2,0.00
5,Alisson,81845,284,https://s.glbimg.com/es/sde/f/2018/05/18/eeece...,4,4.60,Alisson Euler de Freitas Castro,5.4,4,7.35,6,"{'A': 2, 'CA': 1, 'FC': 5, 'FD': 3, 'FS': 6, '...",alisson,5,0.59
6,Leonardo,86973,284,https://s.glbimg.com/es/sde/f/2018/05/21/15d96...,5,2.96,Leonardo Gomes da Conceição Silva,7.5,2,6.32,6,"{'CA': 2, 'FC': 4, 'FD': 3, 'FF': 2, 'FS': 1, ...",leonardo,7,1.49
7,Michel,71920,284,https://s.glbimg.com/es/sde/f/2018/05/21/85cd3...,5,0.80,Michel Ferreira dos Santos,1.1,4,4.04,6,"{'CA': 1, 'FC': 11, 'FD': 2, 'FF': 1, 'GC': 2,...",michel,7,0.68
8,Jean Pyerre,98832,284,https://s.glbimg.com/es/sde/f/2019/05/28/8d48c...,3,5.57,Jean Pyerre Casagrande Silveira Correa,3.1,4,8.60,6,"{'FC': 1, 'FD': 1, 'FF': 5, 'FS': 2, 'G': 1, '...",jean-pyerre,2,-0.71
9,Juninho Capixaba,94968,284,https://s.glbimg.com/es/sde/f/2018/07/25/49d7d...,4,5.48,Luis Antônio da Rocha Júnior,6.7,2,8.08,6,"{'CA': 2, 'FC': 11, 'FS': 11, 'PE': 7, 'RB': 1...",juninho-capixaba,7,1.25


In [7]:
roundN.to_csv("2019_round06_status.csv")